## Atelier 2 : Statistiques des valeurs extrêmes basées sur les maxima journaliers annuels de précipitations

<img src="../images/partners.png" width="50%"/>

Les objectifs d'apprentissage de cet atelier sont les suivants :

1. Comprendre l'extraction des données de précipitations via l'API Google Earth Engine.
2. Le découpage des données pour une région d'intérêt.
3. Le traitement des données de précipitations pour obtenir les maxima annuels dans la région d'intérêt.
4. Ajuster les données à une distribution de Gumbel pour une analyse des valeurs extrêmes.
5. Création du graphique de Gumbel.

#### Il est fortement encouragé de modifier n'importe quelle partie de ce cahier, par exemple la région d'intérêt, la période de temps, les couches de données.

## Importation de l'API Google Earth Engine
Consultez le script "Mise en place du projet Google Earth Engine.ipynb" pour plus d'informations.

In [1]:
import ee
ee.Authenticate()

True

Authentification avec ee :

- Cette commande ouvrira une nouvelle fenêtre vous demandant de vous connecter avec votre compte Google.
- Ensuite, vous devrez autoriser les permissions pour le cahier afin d'accéder à votre compte Google.
- Le processus d'authentification générera un jeton que vous devrez fournir dans une boîte pop-up.
- Le jeton sera sauvegardé et sera valide pendant une semaine.

In [1]:
## Initialisez votre projet en utilisant l'identifiant du projet que vous avez noté lors de
## la création de votre projet Google Earth Engine.
projectID = 'ee-hkvdeveloper'
ee.Initialize(project=projectID)

NameError: name 'ee' is not defined

Maintenant que vous êtes authentifié et avez initialisé votre projet, nous sommes en mesure d'utiliser l'API Google Earth Engine.

## Importer les packages Python

In [3]:
import geemap
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import gumbel_r

Si les packages ne sont pas déjà installés, exécutez :\
!pip install earthengine-api geemap numpy matplotlib scipy

## Sous-échantillonnage des données

Ici, nous définissons la fonction qui nous permet de sous-échantillonner les données pour une région d'intérêt.

In [4]:
# La fonction suivante crée une boîte englobante (Région d'Intérêt) en fonction des latitudes/longitudes fournies. 
# Cela est utile pour limiter la quantité de données que nous récupérons et affichons.

def create_bounding_box(center_lat, center_lon, width, height):
    # Calculer la moitié de la largeur et la moitié de la hauteur.
    half_width = width / 2
    half_height = height / 2

    # Définir les coordonnées de la boîte englobante.
    bounding_box = [
        [center_lon - half_width, center_lat - half_height],  # Coin inférieur gauche
        [center_lon - half_width, center_lat + half_height],  # Coin supérieur gauche
        [center_lon + half_width, center_lat + half_height],  # Coin inférieur droit
        [center_lon + half_width, center_lat - half_height],  # Coin supérieur droit
        [center_lon - half_width, center_lat - half_height],  # Fermeture du polygone
    ]

    return ee.Geometry.Polygon(bounding_box, None, False)

### Spécifier la région d'intérêt, la période de temps et l'ensemble de données de précipitations.

Maintenant, nous spécifions les paramètres d'intérêt pour sous-échantillonner les données.

In [6]:
# N'hésitez pas à changer pour une autre région d'intérêt (ROI)
center_latitude = -18.665695
center_longitude = 35.529564
width = 5 # en degrés
height = 5 # en degrés

# Notez que la boîte englobante peut également être créée en utilisant les latitudes/longitudes minimales et maximales

# Créez la boîte englobante
roi = create_bounding_box(center_latitude, center_longitude, width, height)

# Définissez la période de temps
start_year = 1991
end_year = 2021

# Charger l'ensemble de données de précipitations CHRIPS
# N'hésitez pas à changer pour un autre ensemble de données
chirps = ee.ImageCollection('UCSB-CHG/CHIRPS/DAILY').filterDate(ee.Date(start_year), ee.Date(end_year)).select('precipitation')

# Découpez l'ensemble de données à la région d'intérêt.
chirps_roi = chirps.map(lambda img: img.clip(roi))

## Visualisez les données

Tout d'abord, visualisons l'ensemble de données à l'aide du package geemap. Il est recommandé d'effectuer une étape de visualisation avant de sous-échantillonner les données.

In [7]:
# Créez une carte et ajoutez la couche de précipitations
Map = geemap.Map()
# Centrez la carte en fonction de la région d'intérêt
Map.centerObject(roi, 10)
# Ajoutez la couche de précipitations CHIRPS et attribuez un style
Map.addLayer(chirps_roi, {
    'min': 0,
    'max': 20,
    'opacity': 0.5,
    'palette': ['white', 'blue']
}, 'Mean Precipitation')
Map.addLayerControl()
Map

Map(center=[-18.658919111911636, 35.52956399999995], controls=(WidgetControl(options=['position', 'transparent…

La carte montre que nous avons sélectionné la région d'intérêt à partir de l'ensemble de données CHIRPS.

## Récupérer et traiter les données de précipitations CHIRPS
Ici, nous combinons les étapes précédentes pour :

1. Récupérer les données CHIRPS.
2. Sous-échantillonner les données CHIRPS.
3. Calculer le maximum annuel sur la région d'intérêt.

In [ ]:
# Boucle entre l'année de début (start_year) et l'année de fin (end_year)
yearly_max = []
for year in range(start_year, end_year):  # Exemple : 1981 à 2020
    print(f'Processing year: {year}', end='\r')

    # Spécifiez la date de début (start_date) et la date de fin (end_date) par année
    start_date = f'{year}-01-01'
    end_date = f'{year}-12-31'

    # Récupérer les données CHIRPS via l'API Google Earth Engine
    chirps = ee.ImageCollection('UCSB-CHG/CHIRPS/DAILY')\
                .filterDate(ee.Date(start_date), ee.Date(end_date))\
                .select('precipitation')
    
    # Obtenez la valeur maximale pour l'année
    max_image = chirps.max()

    # Réduisez la région à la région d'intérêt
    max_value = max_image.reduceRegion(ee.Reducer.max(), roi, 1000).get('precipitation').getInfo()

    # Ajoutez la sortie à la liste
    yearly_max.append(max_value)

Les maxima journaliers annuels de précipitations sont collectés et stockés dans la variable yearly_max.

## Analyse des valeurs extrêmes

La prochaine étape consiste à calculer les statistiques des valeurs extrêmes en ajustant une distribution aux maxima journaliers annuels de précipitations. Nous utiliserons la distribution de Gumbel, bien que d'autres types de distributions soient possibles :

![Create a Project](images/extreme_value_analysis.png)
Source: Huyse & Thacker, 2004


## Ajustement des données à une distribution de Gumbel

Ici, nous ajustons la distribution de Gumbel aux données.

In [ ]:
# Ajustez la distribution de Gumbel aux données
params = gumbel_r.fit(yearly_max)
location, scale = params

## Visualisez les résultats

La dernière étape consiste à visualiser les résultats sous la forme d'un graphique de Gumbel

In [ ]:
# Calculer la fonction de répartition cumulative (FRC) pour les valeurs maximales quotidiennes annuelles de précipitations
cdf_values = gumbel_r.cdf(yearly_max, *params)

# Calculer les périodes de retour à partir des valeurs de la fonction de répartition cumulative (FRC)
return_periods = 1 / (1 - cdf_values)

# Triez les données pour le tracé (important pour que les graphiques en ligne aient du sens)
sorted_indices = np.argsort(return_periods)
sorted_return_periods = return_periods[sorted_indices]
sorted_yearly_max = np.array(yearly_max)[sorted_indices]

# Plotting
plt.figure(figsize=(8, 6))
plt.plot(sorted_return_periods, sorted_yearly_max, marker='o', linestyle='-', color='b')
plt.xscale('log')  # Optionnel : Une échelle logarithmique peut rendre le graphique plus facile à interpréter
plt.xlabel('Return Period (years)')
plt.ylabel('Annual Daily Maximum Precipitation (mm)')
plt.title('Gumbel Plot of Annual Daily Maximum Precipitation')
plt.grid(True, which="both", ls="--")
plt.show()

## Vous avez réussi à créer une analyse des valeurs extrêmes de l'occurrence des maxima journaliers annuels de précipitations!

Veuillez essayer de :

- Sélectionner une région d'intérêt différente et comparer les résultats.
- Choisir un ensemble de données de précipitations différent et comparer les résultats.
- Extrapoler le graphique de Gumbel pour des périodes de retour plus élevées.
- Appliquer un ajustement différent, tel que la distribution généralisée des valeurs extrêmes (GEV).